In [15]:
# import the libraries
import pandas as pd
import numpy as np

## Task 1

In [16]:
# path where data is located
data_source_path = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [17]:
# read the webpage into a datafrome
df_list = pd.read_html(data_source_path, header=None)

## Task 2

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [18]:
# store the data in dataframe from first row of the list
df = df_list[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [19]:
# filter the data where Borough is 'Not assigned'
df_filtred = df[df['Borough'] !='Not assigned']
df_filtred.head()
#df_filtred.shape

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


## Task 3

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [20]:
df_aggr = df_filtred.groupby(['Postcode', 'Borough'],as_index=False).agg(lambda x :  ', '.join(x))
df_aggr.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [21]:
df_aggr.shape

(103, 3)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [22]:
df_aggr[df_aggr['Neighbourhood'] =='Not assigned'].head()

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


after the aggrigation there is only one row where Neighbourhood is Not assigned lets replace the value with Borough

In [23]:
 df_aggr['Neighbourhood'] = np.where(df_aggr['Neighbourhood'] == 'Not assigned', df_aggr['Borough'], df_aggr['Neighbourhood'])

In [24]:
df_aggr[df_aggr['Neighbourhood'] =='Not assigned'].head()

,Postcode,Borough,Neighbourhood


after the replacement there is no values in the 'Neighbourhood' colun with not assigned

In [25]:
df_aggr[df_aggr['Postcode'] =='M7A'].head()

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


Borough values is assigned to Neighbourhood

In [26]:
df_aggr.shape

(103, 3)

## Part 2 - Getting latitude and longitude

google API  is not working so I took  the csv file and join it with dataframe from previous assignment

In [27]:
geo_path = 'http://cocl.us/Geospatial_data'
# read the csv into a datafrome
df_geolist = pd.read_csv(geo_path)

In [28]:
df_geolist.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
# join the dataframe from privious assignment with latitude and longitude datafrmae
df_latlong = df_aggr.join(df_geolist.set_index('Postal Code'), on='Postcode')
df_latlong.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
